In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random

# 使用一般 session，不用 requests_cache
session = requests.Session()

# 基本 headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Referer": "https://www.books.com.tw/",
    "Accept-Language": "zh-TW,zh;q=0.9"
}
session.headers.update(headers)

# 防止被封機制
def safe_sleep():
    t = random.uniform(3, 7)
    print(f"⏳ 等待 {t:.2f} 秒...")
    time.sleep(t)

In [2]:
# 分類抓取函式
def get_category(book_url, retries=3):
    for attempt in range(retries):
        try:
            safe_sleep()
            res = session.get(book_url, timeout=15)
            res.raise_for_status()
            html = res.text

            if "限制級商品" in html or "18歲以上會員" in html:
                return "限制級商品（需登入）"
            if "您的連線暫時異常" in html or "Connection is temporarily unavailable" in html:
                return "連線暫時異常"

            soup = BeautifulSoup(html, "html.parser")
            breadcrumb = soup.select("ul#breadcrumb-trail li a")
            if len(breadcrumb) >= 3:
                return breadcrumb[2].text.strip()
            else:
                return "未知分類"
        except Exception as e:
            time.sleep(60)
    return "分類讀取失敗"

In [3]:
# 先測試個別連結是否分類正確
print(get_category("https://www.books.com.tw/products/0011016503?loc=P_0001_015"))
print(get_category("https://www.books.com.tw/products/0011015621?loc=P_0001_016"))

⏳ 等待 6.60 秒...
限制級商品（需登入）
⏳ 等待 6.59 秒...
生活風格


In [4]:
category_count = {}

# 抓排行榜頁
url = "https://www.books.com.tw/web/sys_tdrntb/books/"
res = session.get(url)
soup = BeautifulSoup(res.text, "html.parser")
books = soup.select("li.item")

# 開始爬蟲
seen_links = set()
idx = 1
error_count = 0

import csv

# 建立一個清單來存每本書的資料
book_data = []

for book in books[23:73]:
    title_tag = book.select_one("h4 > a")
    if not title_tag:
        continue

    original_link = title_tag["href"]
    book_id = original_link.split("/products/")[-1].split("?")[0]
    link = f"https://www.books.com.tw/products/{book_id}/{idx}"

    if link in seen_links:
        continue
    seen_links.add(link)

    title = title_tag.text.strip()
    author_tag = book.select_one("ul.msg li:nth-of-type(1) a")
    author = author_tag.text.strip() if author_tag else "未知作者"

    price_tag = book.select_one("ul.msg li.price_a")
    price = "未知價格"
    if price_tag:
        price_text = price_tag.get_text(strip=True)
        if "元" in price_text:
            price = price_text.split("元")[0].split("折")[-1] + "元"

    category = get_category(link)

    if category not in category_count:
        category_count[category] = 1
    else:
        category_count[category] += 1

    if "異常" in category or "失敗" in category:
        error_count += 1
    else:
        error_count = 0

    if error_count >= 1:
        print("⚠️ 連續出錯超過 5 次，暫停 60 秒...")
        time.sleep(60)
        error_count = 0
        category = get_category(link)

    print(f"{idx}. 書名: {title}")
    print(f"    連結: {link}")
    print(f"    作者: {author}")
    print(f"    價格: {price}")
    print(f"    分類: {category}")
    idx += 1

    # 把這筆資料加入 list
    book_data.append([title, author, price, category, link])

⏳ 等待 5.51 秒...
1. 書名: 世界上最透明的故事（日本出版界話題作，只有紙本書可以體驗的感動）
    連結: https://www.books.com.tw/products/0011001466/1
    作者: 杉井光
    價格: 284元
    分類: 文學小說
⏳ 等待 5.97 秒...
2. 書名: 旅者健康守護人
    連結: https://www.books.com.tw/products/0011017700/2
    作者: 未知作者
    價格: 379元
    分類: 醫療保健
⏳ 等待 3.12 秒...
3. 書名: 【中小學生必讀】好好說話超圖解：「換句話說」就能建立好人緣
    連結: https://www.books.com.tw/products/0011007253/3
    作者: 齋藤孝
    價格: 331元
    分類: 童書/青少年文學
⏳ 等待 5.11 秒...
4. 書名: 納瓦爾寶典珍藏版：從白手起家到財務自由，矽谷傳奇創投家的投資哲學與人生智慧
    連結: https://www.books.com.tw/products/0011012422/4
    作者: 艾瑞克．喬根森
    價格: 355元
    分類: 商業理財
⏳ 等待 5.68 秒...
5. 書名: 想見妮‧Nicole妮可寫真書★限量盒裝版【博客來獨家】
    連結: https://www.books.com.tw/products/0011017403/5
    作者: 妮可Nicole
    價格: 優惠價：1200元
    分類: 影視偶像
⏳ 等待 6.34 秒...
6. 書名: 巫師霍爾三部曲（世界奇幻獎終身成就獎得主，生涯代表作）隨書附贈郵票造型書籤卡
    連結: https://www.books.com.tw/products/0010974702/6
    作者: 黛安娜．韋恩．瓊斯
    價格: 950元
    分類: 文學小說
⏳ 等待 4.96 秒...
7. 書名: 養心2：泥魔妖的誘惑（首批附贈作者印簽金句扉頁及五悔角色卡）
    連結: https://www.books.com.tw/products/0011016959/7


In [5]:
# 寫入 CSV 檔案
with open("static.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["書名", "作者", "價格", "分類", "連結"])
    writer.writerows(book_data)

In [6]:
print("\n📊 分類統計結果（依出現次數排序）：")
for cat, count in sorted(category_count.items(), key=lambda x: x[1], reverse=True):
    print(f"分類「{cat}」出現次數：{count}")


📊 分類統計結果（依出現次數排序）：
分類「童書/青少年文學」出現次數：10
分類「商業理財」出現次數：9
分類「心理勵志」出現次數：8
分類「文學小說」出現次數：6
分類「人文社科」出現次數：4
分類「醫療保健」出現次數：2
分類「影視偶像」出現次數：2
分類「語言學習」出現次數：2
分類「國中小參考書」出現次數：2
分類「飲食」出現次數：1
分類「限制級商品（需登入）」出現次數：1
分類「自然科普」出現次數：1
分類「藝術設計」出現次數：1
分類「宗教命理」出現次數：1


In [26]:
import pandas as pd
from datetime import datetime
import csv

# 讀取 static.csv（你的爬蟲輸出）
df = pd.read_csv("static.csv")

# 統計分類出現次數
category_count = df["分類"].value_counts().sort_values(ascending=False)

# 取得當前時間
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# 組合一列：時間 + Top 分類(次數)
row = [now] + [f"{cat}({count})" for cat, count in category_count.items()]

# 檢查欄位長度是否要更新（避免欄位不足）
csv_file = "category_log.csv"
try:
    with open(csv_file, newline='', encoding='utf-8-sig') as f:
        reader = csv.reader(f)
        header = next(reader)
except FileNotFoundError:
    header = ["時間"]
if len(row) > len(header):
    header = ["時間"] + [f"Top{i}" for i in range(1, len(row))]

# 寫入資料（附加）
with open(csv_file, "a", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    # 如果是空檔案，補寫欄位
    if f.tell() == 0:
        writer.writerow(header)
    writer.writerow(row)

print("✅ 已從 static.csv 統計分類並寫入 category_log.csv")

✅ 已從 static.csv 統計分類並寫入 category_log.csv


In [9]:
import requests
import csv
import time
import random

# 自訂搜尋關鍵字，例如可以改成 "python", "小說", "AI", "travel" 等等
query = "python"

# Google Books API 的查詢網址
url = f"https://www.googleapis.com/books/v1/volumes?q={query}&maxResults=20"

# 模擬人類行為，避免 API 請求過快被封
def safe_sleep():
    t = random.uniform(1, 3)
    print(f"⏳ 等待 {t:.2f} 秒...")
    time.sleep(t)

# 發送 API 請求
safe_sleep()
res = requests.get(url)

# 檢查回應是否正常
if res.status_code != 200:
    print("❌ 無法取得資料，HTTP 錯誤碼：", res.status_code)
    exit()

# 將 API 回應轉為 JSON 格式
data = res.json()

# 建立清單存放書籍資料
book_data = []

# 逐筆取出書籍資訊
for item in data.get("items", []):
    info = item.get("volumeInfo", {})
    title = info.get("title", "無標題")
    authors = ", ".join(info.get("authors", ["未知作者"]))
    categories = ", ".join(info.get("categories", ["未知分類"]))
    link = info.get("infoLink", "無連結")
    
    print(f"📘 書名: {title}")
    print(f"    作者: {authors}")
    print(f"    分類: {categories}")
    print(f"    連結: {link}")
    print()

    book_data.append([title, authors, categories, link])

# 將結果寫入 CSV 檔案
with open("api_books.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["書名", "作者", "分類", "連結"])
    writer.writerows(book_data)

print("✅ API 書籍資料已成功寫入 api_books.csv")

⏳ 等待 1.31 秒...
📘 書名: Learning Python
    作者: Mark Lutz
    分類: Computers
    連結: http://books.google.com.tw/books?id=1HxWGezDZcgC&dq=python&hl=&source=gbs_api

📘 書名: Python in a Nutshell
    作者: Alex Martelli
    分類: Computers
    連結: http://books.google.com.tw/books?id=JnR9hQA3SncC&dq=python&hl=&source=gbs_api

📘 書名: Python for Unix and Linux System Administration
    作者: Noah Gift, Jeremy M. Jones
    分類: Computers
    連結: http://books.google.com.tw/books?id=Y7RCCgjmowcC&dq=python&hl=&source=gbs_api

📘 書名: Python Power!
    作者: Matthew A. Telles
    分類: Computers
    連結: http://books.google.com.tw/books?id=754knV_fyf8C&dq=python&hl=&source=gbs_api

📘 書名: Beginning Python
    作者: Magnus Lie Hetland
    分類: Computers
    連結: http://books.google.com.tw/books?id=HMvf12TnJQIC&dq=python&hl=&source=gbs_api

📘 書名: Programming Python
    作者: Mark Lutz
    分類: Computers
    連結: https://play.google.com/store/books/details?id=q8W3WQbNWmkC&source=gbs_api

📘 書名: Python Essential Reference
    作者: 